In [1]:
pip install unsloth transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth.chat_templates import get_chat_template, standardize_sharegpt

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [3]:
updated_weights_address = '/content/drive/MyDrive/finetune/reasoning_fine_tuned_model'

In [4]:
from unsloth import FastLanguageModel
from peft import PeftModel
from transformers import AutoTokenizer

# Load base model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3.2-3b-instruct-unsloth-bnb-4bit",  # Matches your adapter_config.json
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

# Load LoRA adapter
model = PeftModel.from_pretrained(
    model,
    updated_weights_address,
)


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

In [31]:
import re
def cleaner_response(response):
  # match = re.search(r"Question:(.*?)Answer:(.*?)Reason:(.*)Source:(.*?)", response, re.DOTALL)
  # if match:
  #     question = match.group(1).strip()
  #     answer = match.group(2).strip()
  #     reason = match.group(3).strip()
  #     final_response = f"Question: {question}\nAnswer: {answer}\nReason: {reason}"
  # else:
  #     final_response = response  # fallback in case pattern not found
  responses = response.split("assistant")
  # print(responses)
  return responses[-1]
  # return final_response

def make_question_format(example, question):
  return [{"role": "system", "content": "You are a product related question answering assistant. Generate answer based on product's description, specifications, review and previously asked questions. Also write reason as source of your answer(input data line containg answer)/"},
          {"role": "user", "content": f"Product: {example['productName']}\nPrice: {example['price']}\nDescription: {example['description']}\nspecifications: {example['specifications']}\nReviews: {example['revews']}\nPrevious Questions: {example['questionsAnswers']}\nQuestion: {question}"},
          {"role": "assistant", "content": ""}] # Format as list of dicts with role and content


In [32]:
import pandas as pd
questions_files = pd.read_csv("/content/queryProducts_testing.csv")
questions_files.columns = ["productName", "price", "description", "specifications", "revews", "questionsAnswers"]

# Print the actual column names to identify the correct one
print(questions_files.columns)

for index, row in questions_files.iterrows():
  # Access the column using the correct name (e.g., 'product_name')
  # Replace 'productName' with the actual column name from the output above
  print(f"productName: {row['productName']}")
  print(f"price: {row['price']}")  # Assuming 'price' column is correct
  question = input("Enter your question: ")
  inp_prompt = make_question_format(row, question)
  inp_prompt = standardize_sharegpt(inp_prompt)
  input_id = tokenizer.apply_chat_template(inp_prompt, return_tensors="pt").to(model.device)
  output = model.generate(input_ids=input_id, max_new_tokens=200, temperature=0.7, eos_token_id=tokenizer.eos_token_id)
  response = tokenizer.decode(output[0], skip_special_tokens=True)
  # print(response)
  response = cleaner_response(response)
  print(response, "\n\n")

Index(['productName', 'price', 'description', 'specifications', 'revews',
       'questionsAnswers'],
      dtype='object')
productName: MAYBELLINE NEW YORK Call Me Colossal Kit - Mascara+ Kajal+ Bold Liner+ Micellar (Black, 205 g)
price: ₹635
Enter your question: All are waterproof product or not pls tell me the answer?


Source: Previous Questions (Q: All are waterproof product or not pls tell me the answer?, A: Yes it's waterproof)

Answer: Yes, all the products in the Maybelline New York Call Me Colossal Kit are waterproof. The Maybelline Colossal Mascara, Kajal, Bold Liner, and Micellar Water are all waterproof, making them suitable for long-lasting wear and use. 


productName: Men Regular Fit Solid Spread Collar Casual Shirt
price: ₹288
Enter your question: kya iska rang dhone pe nikal jata hai?


Source: [Q: kya iska rang dhone pe nikal jata hai?] 
Answer: No source found for this answer. However, based on the product description, specifications, and reviews, I can provide an a